## MOUNT Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!lscpu |grep 'Model name'
!lscpu |grep 'Core(s) per socket:'
!free -h
!lscpu |grep 'Thread(s) per core'

Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Core(s) per socket:  4
              total        used        free      shared  buff/cache   available
Mem:            51G        935M         47G        1.2M        2.1G         49G
Swap:            0B          0B          0B
Thread(s) per core:  2


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 20 08:26:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## IMPORTS

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import random
import os, sys, time
from PIL import Image
import torchvision.transforms as T
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.ndimage as sc
from PIL import ImageFilter

sys.path.append('/content/drive/MyDrive/ml_epfl/ml_road_segmentation/script/')
#from helper_functions import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## CONSTANTS

In [ ]:
CHECKPOINT_PATH ="/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/current_checkpoint.pt"
BEST_MODEL_PATH ="/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/best_model.pt"
NBR_EPOCHS = 100
BATCH_SIZE = 1
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-6
GAMMA = 1
K_FOLD = 4
VALIDATION_SET_IDX = 0
BATCH_SIZE_VAL = 5
SEED = 0
DIM_IMG_TRAIN = 400

## AUXILIARY FUNCTIONS

In [ ]:
def format_time(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

In [ ]:
def save_ckp(model, state, is_best, checkpoint_path, best_model_path):
    """
    state:            checkpoint we want to save
    is_best:          boolean to indicates if it is the best checkpoint
    checkpoint_path:  path to save checkpoint
    best_model_path:  path to save best model
    """
    torch.save(state, checkpoint_path)
    # if it is a best model, save the model's weights
    if is_best:
        torch.save(model.state_dict(), best_model_path)

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model:           model that we want to load checkpoint parameters into       
    optimizer:       optimizer we defined in previous training
    
    Return model, optimizer, scheduler, epoch value, f1 score
    """
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # Get epoch number, f1_max, scheduler
    epoch = checkpoint['epoch']
    f1_max = checkpoint['f1_max']
    scheduler = checkpoint['scheduler']
    f1_validation = checkpoint['f1_validation']
    acc_validation = checkpoint['acc_validation']
    f1_training = checkpoint['scheduler']
    acc_training = checkpoint['acc_training']


    return model, optimizer, scheduler, epoch, f1_max, f1_validation, acc_validation, f1_training, acc_training

In [ ]:
# FUNCTIONS FOR DATASET AUGMENTATION

#Constants
DIM_IMG = 400
DIM_IMG_CROP=DIM_IMG//2
NB_ROT = 20
ANGLE_ROTATION = 9
    
# Old Rotation
def rotation_old(imgs):
    rot_imgs = [T.functional.rotate(imgs[i], ANGLE_ROTATION*(idx_rot), expand=False)
                for idx_rot in range(NB_ROT)
                for i in range(len(imgs))
                ]
    return rot_imgs

#Rotation with mirroring
def rotation(imgs, angle_rotation, nb_rot):
    rot_imgs = [torch.from_numpy(sc.rotate(imgs[i], angle_rotation*(idx_rot+1), axes =(1,2), reshape=False, mode ='mirror'))
                for idx_rot in range(nb_rot)
                for i in range(len(imgs))
               ]
    return rot_imgs

#Crop and resize
def crop(imgs):
    cropped_imgs = [T.Resize(size=DIM_IMG)(T.FiveCrop(size=DIM_IMG_CROP)(imgs[i])[tuple_i])
                    for tuple_i in range(5)
                    for i in range(len(imgs))
                   ]
    return cropped_imgs

#Composing all transformations    
def compose_all_functions_for_data(imgs, angle_rotation, nb_rot):
    r = imgs + rotation(imgs, angle_rotation, nb_rot)
    return r

def load_train_dataset():
    root_dir = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/data/training/"
    image_dir = root_dir + "images/"
    gt_dir = root_dir + "groundtruth/"
    files = os.listdir(image_dir)
    n = len(files)

    to_tensor = T.ToTensor()
    imgs = [to_tensor(Image.open(image_dir + files[i])) for i in range(n)]
    gt_imgs = [to_tensor(Image.open(gt_dir + files[i])) for i in range(n)]
    return (imgs, gt_imgs)

## UNET MODEL

In [ ]:
def double_conv(nbr_channels_in, nbr_channels_out):
    return nn.Sequential(
      nn.Conv2d(nbr_channels_in, nbr_channels_out, (3,3), padding=(1, 1)),
      nn.BatchNorm2d(nbr_channels_out),
      nn.ReLU(),
      nn.Conv2d(nbr_channels_out, nbr_channels_out, (3,3), padding=(1, 1)),
      nn.ReLU(),
      nn.BatchNorm2d(nbr_channels_out),
)


def one_step_up(x, x_d_i, pre_up_i, up_i):
    return up_i(torch.cat((pre_up_i(x), x_d_i), dim=1))


class u_net(nn.Module):
    def __init__(self):
        super(u_net, self).__init__()

        self.down_pooling = nn.MaxPool2d(2, 2)

        # Convolution Downwards
        self.down_1 = double_conv(6, 64) 
        self.down_2 = double_conv(64, 128)
        self.down_3 = double_conv(128, 256)
        self.down_4 = double_conv(256, 512)
        self.middle =  double_conv(512, 1024)
        
        # Upconvolution
        self.pre_up_1 = nn.ConvTranspose2d(1024, 512, (2, 2), (2, 2))
        self.up_1 = double_conv(512 + 512, 512)
                
        self.pre_up_2 = nn.ConvTranspose2d(512, 256, (2, 2), (2, 2))
        self.up_2 = double_conv(256 + 256, 256)
          
        self.pre_up_3 = nn.ConvTranspose2d(256, 128, (2, 2), (2, 2))
        self.up_3 = double_conv(128 + 128, 128)

        self.pre_up_4 = nn.ConvTranspose2d(128, 64, (2, 2), (2, 2))
        self.up_4 = double_conv(64 + 64, 64)

        self.final_convolution = nn.Conv2d(64, 2, (1,1))
        

    def forward(self, x):
        x_d_1 = self.down_1(x)
        x_d_2 = self.down_2(self.down_pooling(x_d_1))
        x_d_3 = self.down_3(self.down_pooling(x_d_2))
        x_d_4 = self.down_4(self.down_pooling(x_d_3))

        x = self.middle(self.down_pooling(x_d_4))

        x = one_step_up(x, x_d_4, self.pre_up_1, self.up_1)
        x = one_step_up(x, x_d_3, self.pre_up_2, self.up_2)
        x = one_step_up(x, x_d_2, self.pre_up_3, self.up_3)
        x = one_step_up(x, x_d_1, self.pre_up_4, self.up_4)
        
        return self.final_convolution(x)


## DATASET LOADING

In [ ]:
def append_filter(filter_to_apply, images):
    to_tensor = T.ToTensor() 
    toPIL = T.ToPILImage()
    
    concatenated_images = []

    for img in images:
      filtered_img = toPIL(img).filter(filter_to_apply)
      tensored_img = to_tensor(filtered_img)
      stacked_img = torch.cat((img, tensored_img), dim=0)
      concatenated_images.append(stacked_img)
    
    return concatenated_images

In [ ]:
class imagesDataset(Dataset): 
    def __init__(self, K_fold, validation_set_idx, batch_size_val, seed, filter_img, angle_rotation, nb_rot):
        X, Y = load_train_dataset()

        #tresholding ground_truth values
        Y = [(y > 0.5).long() for y in Y]
        shape_y = Y[0].shape
        
        #shuffling
        random.seed(seed)
        idx_list = list(range(len(X)))
        random.shuffle(idx_list)
        random.seed()
        X = [X[idx] for idx in idx_list]
        Y = [Y[idx] for idx in idx_list]
        
        #K_fold separation
        fold_size = len(X) // K_fold
        start_validation_idx = validation_set_idx * fold_size
        end_validation_idx = start_validation_idx + fold_size
        self.X_train = X[slice(0, start_validation_idx)] + X[slice(end_validation_idx, None)]
        self.Y_train = Y[slice(0, start_validation_idx)] + Y[slice(end_validation_idx, None)]
        self.X_validation = X[slice(start_validation_idx, end_validation_idx)]
        self.Y_validation = Y[slice(start_validation_idx, end_validation_idx)]

        #data augmentation
        self.X_train = compose_all_functions_for_data(self.X_train, angle_rotation, nb_rot)
        self.Y_train = compose_all_functions_for_data(self.Y_train, angle_rotation, nb_rot)
        self.X_validation = compose_all_functions_for_data(self.X_validation, angle_rotation, nb_rot)
        self.Y_validation = compose_all_functions_for_data(self.Y_validation, angle_rotation, nb_rot)
        self.n_samples = len(self.X_train)

         #apply filter
        self.X_train = append_filter(filter_img, self.X_train)
        self.X_validation = append_filter(filter_img, self.X_validation)

        #2nd shuffling
        random.seed(seed)
        idx_list = list(range(len(self.X_train)))
        random.shuffle(idx_list)
        random.seed()
        self.X_train = [self.X_train[idx] for idx in idx_list]
        self.Y_train = [self.Y_train[idx] for idx in idx_list]
        
        #casting into tensors
        self.X_train = torch.stack(self.X_train)
        self.X_validation = torch.stack(self.X_validation)
        self.Y_train = torch.reshape(torch.stack(self.Y_train) , (-1, shape_y[1], shape_y[2]))
        self.Y_validation = torch.reshape(torch.stack(self.Y_validation) , (-1, shape_y[1], shape_y[2]))

        #creating dataloader for validation data
        class dataset_validation(Dataset):
            def __init__(s,x,y):
                s.x = x
                s.y = y
                s.size = len(s.x)
            def __getitem__(s, index):
                return s.x[index], s.y[index]
            def __len__(s):
                return s.size
               
        self.validation_data_loader = torch.utils.data.DataLoader(
            dataset_validation(self.X_validation, self.Y_validation),
            batch_size = batch_size_val, shuffle = False)
        
        
    def __getitem__(self, index):
        return self.X_train[index], self.Y_train[index]

    def __len__(self):
        return self.n_samples
    
    def get_validation_dataloader(self):
        return self.validation_data_loader



## TRAINING

In [ ]:
def compute_scores(model, loader, device):
    #computing F1 score on validation data
    tp, fp, fn, tn = 0, 0, 0, 0
    for (data, target) in loader:
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        prediction = torch.argmax(output, dim = 1)
        confusions = prediction / target
        tp += torch.sum(confusions == 1).item()
        fp += torch.sum(confusions == float('inf')).item()
        fn += torch.sum(confusions == 0).item()
        tn += torch.sum(confusions == float("nan")).item()
    f1_score = 2 * tp / (2 * tp + fp + fn)
    accuracy = (tp + tn) / (tp + tn + fp+ fn)
    return f1_score, accuracy

In [ ]:
def train(n_epochs, data_loader, model, optimizer, scheduler, criterion, device, checkpoint_path, best_model_path, f1_init):
    train_loader = data_loader
    validation_loader = train_loader.dataset.get_validation_dataloader()
    f1_max = f1_init
    
    save_f1_validation = []
    save_acc_validation = []
    save_f1_training = []
    save_acc_training = []

    for epoch in range(n_epochs):
        start_time_epoch = time.time()
        loss_list = []
        
        # Train the model for one epoch
        model.train()
        for (data, target) in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        loss_epoch = np.mean(loss_list)

        model.eval()
        #computing scores on validation data
        f1_score_val, accuracy_score_val = compute_scores(model, validation_loader, device)
        save_f1_validation.append(f1_score_val)
        save_acc_validation.append(accuracy_score_val)

        #computing scores on training data
        f1_score_train, accuracy_score_train = compute_scores(model, train_loader, device)
        save_f1_training.append(f1_score_train)
        save_acc_training.append(accuracy_score_train)

        end_time_epoch = time.time()

        # Prepare saving of the model
        checkpoint = {
            'epoch': epoch,
            'f1_max': f1_max,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler,
            'f1_validation' : save_f1_validation,
            'acc_validation' : save_acc_validation,
            'f1_training' : save_f1_training,
            'acc_training' : save_acc_training,
        }

        if f1_score_val > f1_max:
            save_ckp(model, checkpoint, True, checkpoint_path, best_model_path)
            f1_max = f1_score_val
        
        save_ckp(model, checkpoint, False, checkpoint_path, best_model_path)

        #print(f"Epoch {epoch} || Loss:{loss_epoch:.6f} || Training F1 {f1_score_train:.6f} || Training Accuracy {accuracy_score_train:.6f} || Validation F1 {f1_score_val:.6f} || Validation Accuracy {accuracy_score_val:.6f} || Max F1 {f1_max:.6f} || LR {scheduler.get_last_lr()[0]:.6f} || Duration {format_time(end_time_epoch-start_time_epoch)}"+"\n")  
        scheduler.step()

## K-FOLD CROSS VALIDATION (hyper-parameters)

In [ ]:
"""CHECKPOINT_K_FOLD_PATH_ROOT ="/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/k_folds_checkpoints/"
BEST_MODEL_K_FOLD_PATH_ROOT ="/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/k_folds_checkpoints/"

# Constants K-Fold
K_FOLD = 4
LEARNING_RATE_K_FOLD = 1e-3
WEIGHT_DECAY_K_FOLD = 1e-6
MOMENTUM_SGD_K_FOLD = 0.99
GAMMA_K_FOLD = 1
SEED_K_FOLD = 0
NBR_EPOCHS_K_FOLD = 15
BATCH_SIZE_VAL_K_FOLD = 5
BATCH_SIZE_K_FOLD = 1
FILTER = ImageFilter.FIND_EDGES

OPTIMIZERS = ["SGD_", "ADAM_", "ADAM_W_"]

for opti_name in OPTIMIZERS:
    for idx_validaton_set in range(K_FOLD): 
        print(f'Optimizer {opti_name[:-1]} - fold {idx_validaton_set}')
        
        #creating model
        model =  u_net().to(device)
        
        #creating optimizer
        optimizer = None
        if opti_name == "SGD_":
            optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE_K_FOLD, weight_decay=WEIGHT_DECAY_K_FOLD, momentum=MOMENTUM_SGD_K_FOLD) 
        elif opti_name == "ADAM_":
            optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE_K_FOLD, weight_decay=WEIGHT_DECAY_K_FOLD)
        elif opti_name == "ADAM_W_":
            optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE_K_FOLD, weight_decay=WEIGHT_DECAY_K_FOLD)

        scheduler = ExponentialLR(optimizer, GAMMA_K_FOLD)
        criterion = nn.CrossEntropyLoss()
        dataset = imagesDataset(K_FOLD, idx_validaton_set, BATCH_SIZE_VAL_K_FOLD, SEED_K_FOLD, FILTER)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE_K_FOLD, shuffle = True)
        validation_dataloader = dataset.get_validation_dataloader()

        # Give correct paths
        CHECKPOINT_K_FOLD_PATH = CHECKPOINT_K_FOLD_PATH_ROOT + opti_name + 'fold_' + str(idx_validaton_set) +'_all_data_final.pt' 
        BEST_MODEL_K_FOLD_PATH = BEST_MODEL_K_FOLD_PATH_ROOT + opti_name + 'fold_' + str(idx_validaton_set) +'_model_final.pt'
        train(NBR_EPOCHS_K_FOLD, train_loader, model, optimizer, scheduler, criterion, device, CHECKPOINT_K_FOLD_PATH, BEST_MODEL_K_FOLD_PATH, f1_init = 0)"""

Optimizer SGD - fold 0
Optimizer SGD - fold 1
Optimizer SGD - fold 2
Optimizer SGD - fold 3
Optimizer ADAM - fold 0
Optimizer ADAM - fold 1
Optimizer ADAM - fold 2
Optimizer ADAM - fold 3
Optimizer ADAM_W - fold 0
Optimizer ADAM_W - fold 1
Optimizer ADAM_W - fold 2
Optimizer ADAM_W - fold 3


## K-FOLD SGD FINAL (for bagging)

In [ ]:
K_FOLD_ROOT_PATH_LOADING = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/k_folds_checkpoints/"
K_FOLD_ROOT_PATH_WRITTING = "/content/drive/MyDrive/ml_epfl/ml_road_segmentation/checkpoint/k_folds_bagging/"

# Constants K-Fold
K_FOLD = 4
LEARNING_RATE_K_FOLD = 1e-3
WEIGHT_DECAY_K_FOLD = 1e-6
MOMENTUM_SGD_K_FOLD = 0.99
GAMMA_K_FOLD = 1
SEED_K_FOLD = 0
NBR_EPOCHS_K_FOLD = 30
BATCH_SIZE_VAL_K_FOLD = 5
BATCH_SIZE_K_FOLD = 1
FILTER = ImageFilter.FIND_EDGES
OPTIMIZERS = ["SGD_"]

for opti_name in OPTIMIZERS:
    for idx_validaton_set in range(K_FOLD): 
        print(f'Optimizer {opti_name[:-1]} - fold {idx_validaton_set}')
        #defining paths
        #CHECKPOINT_PATH_LOADING = K_FOLD_ROOT_PATH_LOADING + opti_name + 'fold_' + str(idx_validaton_set) +'_all_data_final.pt'
        CHECKPOINT_K_FOLD_PATH = K_FOLD_ROOT_PATH_WRITTING + opti_name + 'fold_' + str(idx_validaton_set) +'_all_data_bagging.pt' 
        BEST_MODEL_K_FOLD_PATH = K_FOLD_ROOT_PATH_WRITTING + opti_name + 'fold_' + str(idx_validaton_set) +'_model_bagging.pt'

        #loading model
        checkpoint = torch.load(CHECKPOINT_K_FOLD_PATH)
        f1_max = checkpoint['f1_max']
        model =  u_net().to(device)
        model.load_state_dict(checkpoint["state_dict"])

        #defining other parameters
        optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE_K_FOLD, weight_decay=WEIGHT_DECAY_K_FOLD, momentum=MOMENTUM_SGD_K_FOLD) 
        scheduler = ExponentialLR(optimizer, GAMMA_K_FOLD)
        criterion = nn.CrossEntropyLoss()

        #loading dataset
        dataset = imagesDataset(K_FOLD, idx_validaton_set, BATCH_SIZE_VAL_K_FOLD, SEED_K_FOLD, FILTER)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE_K_FOLD, shuffle = True)
        validation_dataloader = dataset.get_validation_dataloader()

        #resuming training
        train(NBR_EPOCHS_K_FOLD, train_loader, model, optimizer, scheduler, criterion, device, CHECKPOINT_K_FOLD_PATH, BEST_MODEL_K_FOLD_PATH, f1_init = f1_max)

Optimizer SGD - fold 0
